https://github.com/leelabcnbc/unsup-pytorch/issues/2

In [1]:
import numpy as np
from torch import FloatTensor
from torch.autograd import Variable
from torch.nn.functional import mse_loss, conv2d

In [2]:
from torch.nn import ConvTranspose2d

In [3]:
from scipy.signal import convolve

In [4]:
import torch

In [5]:
_ = torch.manual_seed(0)

In [6]:
num_channel_code = 4
kernel_size = 9
input_this = FloatTensor(1, num_channel_code, 17, 17)
_ = input_this.normal_(0, 1)

In [7]:
def forward():
    code_to_image_layer = ConvTranspose2d(num_channel_code, 1, kernel_size, bias=False)
    input_var = Variable(input_this, requires_grad=True)
    output_now = code_to_image_layer(input_var).data.numpy()[0,0]
    print(output_now.shape)
    
    # try to replicate the result using naive `convolve` in `scipy.signal`
    weight_this = code_to_image_layer.weight.data.numpy()[:,0]
    print(weight_this.shape)
    input_this_np = input_var.data.numpy()[0]
    print(input_this_np.shape)
    
    output_ref = np.zeros_like(output_now, dtype=np.float64)
    assert weight_this.shape[0] == input_this_np.shape[0]
    for weight_this_slice, input_this_np_slice in zip(weight_this, input_this_np):
        # these two are the same.
#         output_ref += convolve(input_this_np_slice, weight_this_slice)
        output_ref += convolve(weight_this_slice, input_this_np_slice)
    assert output_now.shape == output_ref.shape
    print(abs(output_now-output_ref).max())
    assert abs(output_now-output_ref).max()<1e-6

In [8]:
forward()

(25, 25)
(4, 9, 9)
(4, 17, 17)
6.25848770142e-07
